# Encuesta continua de hogares 2020

### Cuánto afecta el nivel eductativo y la cantidad de hijos a las posibilidades de inserción laboral de una mujer

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
df = pd.read_spss('HyP_2020_Terceros.sav')

df.to_csv('encuesta.csv', index=False)


df = df.rename(columns={
    'numero': 'id_hogar',
    'e26': 'sexo',
    'e563': 'genero',
    'e29_6': 'ascendencia',
    'e30': 'parentesco',
    'pobpcoac': 'estado_laboral',
    })

df = df[['id_hogar', 'sexo', 'genero', 'ascendencia', 'parentesco', 'estado_laboral', 'nper']]

In [3]:
df.sample(5)

,id_hogar,sexo,genero,ascendencia,parentesco,estado_laboral,nper
23300,19715,2.0,Mujer,Afro o negra,Jefe/a de hogar,Ocupados,1.0
125194,8210,1.0,Varón,Blanca,Jefe/a de hogar,Ocupados,1.0
11628,14542,2.0,Mujer,Blanca,Hijo/a solo del jefe/a,Ocupados,2.0
87062,45961,1.0,Varón,Blanca,Jefe/a de hogar,Ocupados,1.0
64672,36643,1.0,Varón,Asiática o amarilla,Jefe/a de hogar,"Inactivo, otro",1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145166 entries, 0 to 145165
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   id_hogar        145166 non-null  object  
 1   sexo            145166 non-null  float64 
 2   genero          145166 non-null  category
 3   ascendencia     145166 non-null  category
 4   parentesco      145166 non-null  category
 5   estado_laboral  145166 non-null  category
 6   nper            145166 non-null  float64 
dtypes: category(4), float64(2), object(1)
memory usage: 3.9+ MB


In [5]:
df['estado_laboral'].value_counts() / len(df['estado_laboral']) * 100

Ocupados                                      43.915242
Inactivo, jubilado                            16.902029
Menor 14 años                                 15.480898
Inactivo, estudiante                           8.152735
Inactivo, realiza los quehaceres del hogar     5.433779
Inactivo, pensionista                          3.998870
Desocupados propiamente dichos                 3.600705
Inactivo, otro                                 1.193806
Desocupados BT1V                               0.688867
Inactivo, rentista                             0.340989
Desocupados en seguro de desempleo             0.292079
Name: estado_laboral, dtype: float64

## Género

In [6]:
df['genero'].value_counts()

Mujer                    69037
Varón                    57429
0.0                      18571
Sin dato                    78
Mujer trans                 29
Varón trans                 14
Otra                         6
No sabe / No contesta        2
Name: genero, dtype: int64

## Ascendencia

In [7]:
asc_princ_porcentage = df['ascendencia'].value_counts() / len(df['ascendencia']) * 100

asc_princ_porcentage

Blanca                 69.510767
Asiática o amarilla    23.261645
Afro o negra            3.583484
Sin dato                2.384856
Indígena                1.244782
Otra                    0.014466
Name: ascendencia, dtype: float64

## Composición del hogar

In [8]:
df['id_hogar'].value_counts()


56535    16
9287     13
14619    13
23586    13
13445    12
         ..
2648      1
26480     1
26482     1
26496     1
9998      1
Name: id_hogar, Length: 58594, dtype: int64

In [9]:
df['parentesco'].value_counts() 

Jefe/a de hogar                            58021
Esposo/a o compañero/a                     31354
Hijo/a de ambos                            27083
Hijo/a solo del jefe/a                     15987
Nieto/a                                     4233
Hijo/a solo del esposo/a compañero/a        2288
Padre/madre                                 1601
Hermano/a                                   1070
Otro pariente                                888
Yerno/nuera                                  875
Otro no pariente                             865
Suegro/a                                     412
Sin dato                                     316
Cuñado/a                                     149
Servicio doméstico o familiar del mismo       24
Name: parentesco, dtype: int64

In [10]:
familia = pd.pivot_table(df, index='id_hogar', columns='parentesco',values=['nper'], aggfunc='count')

familia['totales'] = familia.apply(lambda x:x.sum(), axis=1)

familia = familia.copy().sort_values(by='totales', ascending=False)

familia

nper                                                   \
parentesco Cuñado/a Esposo/a o compañero/a Hermano/a Hijo/a de ambos   
id_hogar                                                               
56535             0                      1         0              10   
14619             0                      0         0               0   
9287              0                      0         0               0   
23586             0                      0         0               0   
52853             0                      1         0               3   
...             ...                    ...       ...             ...   
37352             0                      0         0               0   
37338             0                      0         0               0   
37337             0                      0         0               0   
37336             0                      0         0               0   
36366             0                      0         0               0   

                                                                        \
parentesco Hijo/a solo del esposo/a compañero/a Hijo/a solo del jefe/a   
id_hogar                                                                 
56535                                         0                      0   
14619                                         0                      4   
9287                                          0                      5   
23586                                         0                      6   
52853                                         0                      0   
...                                         ...                    ...   
37352                                         0                      0   
37338                                         0                      0   
37337                                         0                      0   
37336                                         0                      0   
36366                                         0                      0   

                                                                               \
parentesco Jefe/a de hogar Nieto/a Otro no pariente Otro pariente Padre/madre   
id_hogar                                                                        
56535                    1       4                0             0           0   
14619                    1       4                2             0           0   
9287                     1       6                0             0           0   
23586                    1       3                1             0           0   
52853                    1       5                0             0           0   
...                    ...     ...              ...           ...         ...   
37352                    1       0                0             0           0   
37338                    1       0                0             0           0   
37337                    1       0                0             0           0   
37336                    1       0                0             0           0   
36366                    1       0                0             0           0   

                                                                      \
parentesco Servicio doméstico o familiar del mismo Sin dato Suegro/a   
id_hogar                                                               
56535                                            0        0        0   
14619                                            0        0        0   
9287                                             0        0        0   
23586                                            0        2        0   
52853                                            0        0        0   
...                                            ...      ...      ...   
37352                                            0        0        0   
37338                                            0        0        0   
37337                                     

In [11]:
familia['totales'].value_counts().sort_index()

1     15749
2     18429
3     11547
4      8634
5      2870
6       887
7       267
8       134
9        44
10       11
11        8
12       10
13        3
16        1
Name: totales, dtype: int64

In [12]:
df['parentesco_num'] = df['parentesco'].cat.codes

In [13]:
df[['parentesco_num','parentesco']].value_counts().sort_index()

parentesco_num  parentesco                             
0               Cuñado/a                                     149
1               Esposo/a o compañero/a                     31354
2               Hermano/a                                   1070
3               Hijo/a de ambos                            27083
4               Hijo/a solo del esposo/a compañero/a        2288
5               Hijo/a solo del jefe/a                     15987
6               Jefe/a de hogar                            58021
7               Nieto/a                                     4233
8               Otro no pariente                             865
9               Otro pariente                                888
10              Padre/madre                                 1601
11              Servicio doméstico o familiar del mismo       24
12              Sin dato                                     316
13              Suegro/a                                     412
14              Yerno/nuera       

In [14]:
mask = df['parentesco_num'].isin(list(range(3,7)))

df = df[mask]

df['parentesco_num'].value_counts()

6    58021
3    27083
5    15987
4     2288
Name: parentesco_num, dtype: int64

In [15]:
df

,id_hogar,sexo,genero,ascendencia,parentesco,estado_laboral,nper,parentesco_num
0,100,2.0,Mujer,Blanca,Jefe/a de hogar,Ocupados,1.0,6
2,100,1.0,0.0,Blanca,Hijo/a de ambos,Menor 14 años,3.0,3
3,1000,1.0,Varón,Blanca,Jefe/a de hogar,"Inactivo, jubilado",1.0,6
4,1000,2.0,Mujer,Blanca,Hijo/a solo del jefe/a,"Inactivo, pensionista",2.0,5
5,10000,1.0,Varón,Blanca,Jefe/a de hogar,"Inactivo, jubilado",1.0,6
...,...,...,...,...,...,...,...,...
145161,9992,2.0,Mujer,Blanca,Jefe/a de hogar,Ocupados,1.0,6
145162,9993,1.0,Varón,Blanca,Jefe/a de hogar,Ocupados,1.0,6
145163,9994,1.0,Varón,Blanca,Jefe/a de hogar,Ocupados,1.0,6
145164,9996,2.0,Mujer,Blanca,Jefe/a de hogar,"Inactivo, pensionista",1.0,6


In [19]:
df[['id_hogar', 'parentesco_num']].groupby(by='id_hogar').count().sort_values(by='parentesco_num', ascending=False)

,parentesco_num
id_hogar,
56535,11
26497,9
26507,9
26511,9
26514,9
...,...
33027,1
33028,1
33029,1
